In [1]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
%load_ext autoreload
%autoreload 2

In [2]:
def call_spread (S0, K1, K2, T, r, vol):
    C1=opt.black_scholes(r, S0, K1, T, vol, type="c")
    C2=opt.black_scholes(r, S0, K2, T, vol, type="c")
    return C1-C2

In [3]:
def down_and_in_put(S0, K, H, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        BARRIER=False
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
            if St<=H:
                BARRIER=True
        
        if BARRIER:
            CT=max(0,K-St)
        else:
            CT=0
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)
    
    return C0

### Knock-in Reverse Convertible

In [4]:
def knock_in_reverse_convertible(A, S0, H, c, t, T, r, vol, N, M):     ##t in months
    ## Renvoie la valeur d'un coupon en utilisant c la valeur d'une obligation basique
    obs=T/(t/12)
    V=0
    final_discount=1/((1+c)**obs)
    for i in range (1,int(obs)+1):
        V+=1/((1+c)**i)

    P=down_and_in_put(S0, S0, H, T, r, vol, N, M)
    
    return ((1-final_discount+P/A)/V)*100     ##pour avoir le résultat en pourcentage

In [5]:
knock_in_reverse_convertible(100, 100, 80, 0.0175, 6, 2, 0.04, 0.2, 100, 1000)

3.5762878150520816

### Autocallable

In [6]:
def autocallable (A, S0, H1, H2, H3, c, t, T, r, vol, N, M):
    ## H1 est la barrière basse de knock-in, H2 est la barrière pour les coupons
    ## et H3 est la barrière haute (de knock-out avec rebate)
        
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)
    obs=int(N/(t/12))

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        knocked_out=False
        knocked_in=False
        CT=0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
            
            ##Down and in put option à H1
            if St<H1:
                knocked_in=True

            if (j+1)%obs==0:

                if St>=H3:
                    knocked_out=True
                    break
                
                if St>H2:
                    CT+=call_spread(St, H2-c*100, H2, (T - (j+1) * dt), r, vol)
        
        if knocked_in:
            #l'investisseur reçoit son montant proportionnel à la chute de l'action
            CT+= A-max(0,S0-St)
        
        if knocked_out:
            #la barrière a été atteinte et l'investisseur reçoit son montant investi avant échéance
            CT+=A
        
        if not knocked_out and not knocked_in : 
            #On arrive à échéance et l'investisseur reçoit le montant investi même si la barrière de knock-out n'a pas été atteinte
            CT+=A
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    return C0

In [7]:
autocallable(100, 100, 80, 90, 110, 0.04, 6, 2, 0.02, 0.2, 100, 1000)

87.85433323138874

In [ ]:
def autocallable_unconditional (A, S0, H1, H2, H3, c, t, T, r, vol, N, M):
    ## H1 est la barrière basse de knock-in, H2 est la barrière pour les coupons
    ## et H3 est la barrière haute (de knock-out avec rebate)
        
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)
    obs=int(N/(t/12))

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        knocked_out=False
        knocked_in=False
        CT=0
        total_coupon=0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
            
            ##Down and in put option à H1
            if St<H1:
                knocked_in=True

            if (j+1)%obs==0:
                total_coupon+=c

                if St>=H3:
                    knocked_out=True
                    CT=A+A*total_coupon
                    break
                
                CT+=call_spread(S0, H2-c*100, H2, (T - (j+1) * dt), r, vol)
        
        if knocked_in:
            CT+= -max(0,S0-St)
        
        if not knocked_out:
            CT+=A+A*total_coupon  
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    return C0